In [56]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [57]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [58]:
os.getcwd()

'/search/odin/chenghuige/sogou/pikachu/projects/feed/rank/src/jupyter'

In [95]:
# 全量
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# 8
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/19'
# 16
info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1'
# info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/tuwen/11'

# info_dir = f'/search/odin/yuwenmengke/pikachu/projects/feed/rank/src/v25_tw_mktest_models_0430/tuwen_0430_tw_withScoreemb_withUseractemb_withDevemb_withDistrib_withKw_noSrch/infos'

hour = 2020081622
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df['mark'] = 'tuwen'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
if 'product_data' in df.columns:
  df = df[df.product_data=='sgsapp']
df.head()

/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081622/valid.csv


,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,mark
0,0,0,0,4,0,2,-2641845729829102622,3057,20730h042pug19,0,...,0,1.0,0,sgsapp,0.03584691,0.18594688,0.19578531,2020081622,0,tuwen
1,0,0,0,12,0,1,5842802009574613944,3057,sf_478365372694102016,0,...,0,1.0,0,sgsapp,0.10569633,0.26962218,0.46022263,2020081622,0,tuwen
2,0,0,0,19,0,2,7933546124364592930,3116,20810e041GQuOY,178,...,1,1.0,0,sgsapp,0.05164630,0.26794282,0.16737556,2020081622,178,tuwen
3,0,0,0,2,0,6,667360749037357052,update,20816b0K2Yf5vP,0,...,0,1.0,0,sgsapp,0.01432269,0.07566941,0.28038470,2020081622,0,tuwen
4,0,0,0,7,0,1,2409286417499251823,1445,open_doc_prod36373979,0,...,0,1.0,0,sgsapp,0.01517263,0.06481814,0.40584900,2020081622,0,tuwen


In [96]:
df.user_active

0          2
1          2
2          2
3          2
4          1
          ..
2108305    0
2108356    2
2108407    2
2108458    2
2108509    2
Name: user_active, Length: 986745, dtype: int64

In [97]:
df.hour

0          2020081622
1          2020081622
2          2020081622
3          2020081622
4          2020081622
              ...    
2108305    2020081622
2108356    2020081622
2108407    2020081622
2108458    2020081622
2108509    2020081622
Name: hour, Length: 986745, dtype: int64

In [98]:
df['click'] = (df.dur != 0).astype(int)

In [99]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 1060/1060 [00:39<00:00, 27.20it/s]


In [100]:
df['mark'] = df.title.apply(lambda x: '苟晶' in x)

In [101]:
df['click'] = (df.duration != 0).astype(int)

In [102]:
len(df[df.mark==True])

0

In [103]:
len(df[(df.mark==True)&(df.duration!=0)])

0

In [104]:
df_doc = df.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})

In [105]:
df_doc['ctr'] = df_doc.clicks / df_doc.shows

In [106]:
df_doc

,docid,shows,clicks,ctr
0,09fc3dd81d66887a8df94bea3bbddf7c,8,2,0.25
1,15f3e2b88b172b37446ebeca5f721666,2,1,0.50
2,197170044C2nuf,1,1,1.00
3,19727m044afrjv,4,0,0.00
4,1980220443tQfd,6,3,0.50
...,...,...,...,...
105952,sv_210167286,4,0,0.00
105953,sv_210167643,8,0,0.00
105954,sv_210169956,1,0,0.00
105955,sv_210172098,20,3,0.15


In [107]:
df['dur2'] = df.dur.apply(lambda x: x if x > 0 else 0)
df['click2'] = df.dur.apply(lambda x: 1 if x > 0 else 0)

In [108]:
df_doc2 = df.groupby('docid', as_index=False)["dur2"].agg({"tdur" : sum})
df_doc3 = df.groupby('docid', as_index=False)["click2"].agg({"clicks2" : sum})

In [109]:
df_doc2

,docid,tdur
0,09fc3dd81d66887a8df94bea3bbddf7c,497
1,15f3e2b88b172b37446ebeca5f721666,287
2,197170044C2nuf,379
3,19727m044afrjv,0
4,1980220443tQfd,580
...,...,...
105952,sv_210167286,0
105953,sv_210167643,0
105954,sv_210169956,0
105955,sv_210172098,403


In [110]:
df_doc3

,docid,clicks2
0,09fc3dd81d66887a8df94bea3bbddf7c,2
1,15f3e2b88b172b37446ebeca5f721666,1
2,197170044C2nuf,1
3,19727m044afrjv,0
4,1980220443tQfd,3
...,...,...
105952,sv_210167286,0
105953,sv_210167643,0
105954,sv_210169956,0
105955,sv_210172098,3


In [111]:
df_doc = df_doc.merge(df_doc2, on='docid', how='left')
df_doc = df_doc.merge(df_doc3, on='docid', how='left')

In [112]:
df_doc

,docid,shows,clicks,ctr,tdur,clicks2
0,09fc3dd81d66887a8df94bea3bbddf7c,8,2,0.25,497,2
1,15f3e2b88b172b37446ebeca5f721666,2,1,0.50,287,1
2,197170044C2nuf,1,1,1.00,379,1
3,19727m044afrjv,4,0,0.00,0,0
4,1980220443tQfd,6,3,0.50,580,3
...,...,...,...,...,...,...
105952,sv_210167286,4,0,0.00,0,0
105953,sv_210167643,8,0,0.00,0,0
105954,sv_210169956,1,0,0.00,0,0
105955,sv_210172098,20,3,0.15,403,3


In [113]:
df_doc['mdur'] = df_doc.tdur / df_doc.clicks2
df_doc = df_doc.fillna(0)
df_doc.mdur = df_doc.mdur.astype(int)
df_doc

,docid,shows,clicks,ctr,tdur,clicks2,mdur
0,09fc3dd81d66887a8df94bea3bbddf7c,8,2,0.25,497,2,248
1,15f3e2b88b172b37446ebeca5f721666,2,1,0.50,287,1,287
2,197170044C2nuf,1,1,1.00,379,1,379
3,19727m044afrjv,4,0,0.00,0,0,0
4,1980220443tQfd,6,3,0.50,580,3,193
...,...,...,...,...,...,...,...
105952,sv_210167286,4,0,0.00,0,0,0
105953,sv_210167643,8,0,0.00,0,0,0
105954,sv_210169956,1,0,0.00,0,0,0
105955,sv_210172098,20,3,0.15,403,3,134


In [114]:
df = df.merge(df_doc, on='docid', how='left')

In [115]:
df

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,mark,title,dur2,click2,shows,clicks,ctr,tdur,clicks2,mdur
0,0,0,0,4,0,2,-2641845729829102622,3057,20730h042pug19,0,...,False,农村不起眼的暴利行业，一天赚5000闷声发大财，埋头干几年退休,0,0,48,7,0.14583333,595,4,148
1,0,0,0,12,0,1,5842802009574613944,3057,sf_478365372694102016,0,...,False,3首最好听歌曲《不要走》《万爱千恩》《送亲》值得收藏,0,0,30,5,0.16666667,1857,5,371
2,0,0,0,19,0,2,7933546124364592930,3116,20810e041GQuOY,178,...,False,装备出意外，张怡宁用十块钱球拍虐翻对手，连教练都看不下去了,178,1,202,39,0.19306931,4569,31,147
3,0,0,0,2,0,6,667360749037357052,update,20816b0K2Yf5vP,0,...,False,胖妹家的早餐真丰富，看看都吃的啥，看的口水直流。,0,0,258,47,0.18217054,9958,43,231
4,0,0,0,7,0,1,2409286417499251823,1445,open_doc_prod36373979,0,...,False,二胎时代：新员工不认识老板娘，拦着不让进，下秒老板娘霸气了,0,0,2,0,0.00000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986740,0,0,0,7,0,2,-1431340396460958305,5153,20814d0K3OVZDc,0,...,False,战舰世界：一级战备，保卫领海寸步不让，8月13日海战战斗报告,0,0,1598,223,0.13954944,34675,181,191
986741,0,0,0,13,0,1,-2803269093125291825,3039,20708k0K4ENNYD,95,...,False,米饭达人：吴京肉沫茄子下三碗饭，林雪连吃五碗，梁家辉开始表演,95,1,6,1,0.16666667,95,1,95
986742,0,0,0,12,0,1,-88796746097124616,3116,20711b0K2i8yk4,0,...,False,影视中爆笑借钱名场面：大鹏竟向和尚借钱，和尚的回答亮了，绝!,0,0,65,15,0.23076923,3160,15,210
986743,0,0,0,12,0,20,4992875076576397920,3065,20802i0K2cEbLX,411,...,False,八仙：贪官得一宝贝，能点物成金，结果不小心把爱人也变成了金子,411,1,6,1,0.16666667,411,1,411


In [116]:
df_ = df.docid.value_counts().reset_index().head(20)
df_['title'] = df_['index'].apply(lambda x: titles.get(x, ''))
df_

,index,docid,title
0,2081600K4o4SgA,8142,趁着你老婆没看见，赶紧看完吧。带上耳机最佳
1,20815d043j43Dg,7347,恐怖片：小伙将自己化妆成死尸，本想吓唬回家的室友，结果救了自己一命
2,20815i0K0Dky56,5134,难得一见激烈战斗蟒蛇发飙张开大嘴狠咬狮子见激烈战斗!
3,20816e0Q1gbo4w,4730,龟头敏感早泄射精快怎么办，陈梅医生分享4个原因和解决方法
4,20815k0K3kyHmZ,3819,"邓超演唱《娘娘我错了》,“娘娘”惊喜登台他懵了!明星蒙圈集锦"
5,20815g0K3TyoOw,3671,影视：穷小子混成亿万总裁，参加同学聚会偶遇初恋，好戏要开始
6,open_doc_prod38775855,3527,港珠澳大桥隧道有46米深，万一漏水怎么逃生?看完实在佩服
7,20809c0R46SwzX,3242,张怡宁对阵福原爱，一旁的教练急得不行：你让两球啊，她都要哭了
8,20527i0K43HFxB,3164,假如越狱分段位，白展堂直接封神：老邢，再不开门我就出去了
9,20814j043wYJx3,2914,过气演员入住鬼宅，撞上阴兵借道，只能贴着墙根屏住呼吸


In [117]:
def top_recall(df, abid):
  df_ = df[df.abtestid==abid]
  x = df_.docid.value_counts().head(20).reset_index()
  x = x.rename(columns={'index': 'docid', 'docid': 'count'})
  df_doc = df_.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})
  df_doc['ctr'] = df_doc.clicks / df_doc.shows
  x = x.merge(df_doc, on='docid', how='left')
  x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
  print(sum(x.clicks))
  print(sum(x.clicks) / sum(x.shows))
  return x

In [118]:
top_recall(df, 5)

733
0.2164796219728293


,docid,count,shows,clicks,ctr,title
0,2081600K4o4SgA,366,366,111,0.30327869,趁着你老婆没看见，赶紧看完吧。带上耳机最佳
1,20815d043j43Dg,352,352,111,0.31534091,恐怖片：小伙将自己化妆成死尸，本想吓唬回家的室友，结果救了自己一命
2,20815i0K0Dky56,247,247,89,0.36032389,难得一见激烈战斗蟒蛇发飙张开大嘴狠咬狮子见激烈战斗!
3,20816e0Q1gbo4w,230,230,104,0.45217391,龟头敏感早泄射精快怎么办，陈梅医生分享4个原因和解决方法
4,20815k0K3kyHmZ,204,204,30,0.14705882,"邓超演唱《娘娘我错了》,“娘娘”惊喜登台他懵了!明星蒙圈集锦"
5,open_doc_prod38775855,193,193,15,0.07772021,港珠澳大桥隧道有46米深，万一漏水怎么逃生?看完实在佩服
6,20815g0K3TyoOw,168,168,41,0.24404762,影视：穷小子混成亿万总裁，参加同学聚会偶遇初恋，好戏要开始
7,open_doc_prod38511337,151,151,20,0.13245033,农民发现自家麦田被毁，用无人机观察后，暗自庆幸自己没有进去
8,20814g0Q0odUdB,145,145,28,0.19310345,中国将上万条“大鱼”养在三峡，如今变成了什么样?看完发出赞叹
9,2081430K0bWAoP,144,144,31,0.21527778,我心中最“肮脏”的越南电影，说破了成人的私生活，看完脊背发凉


In [119]:
top_recall(df, 16)

685
0.19454700369213293


,docid,count,shows,clicks,ctr,title
0,2081600K4o4SgA,413,413,116,0.28087167,趁着你老婆没看见，赶紧看完吧。带上耳机最佳
1,20815d043j43Dg,349,349,95,0.27220630,恐怖片：小伙将自己化妆成死尸，本想吓唬回家的室友，结果救了自己一命
2,20815i0K0Dky56,254,254,89,0.35039370,难得一见激烈战斗蟒蛇发飙张开大嘴狠咬狮子见激烈战斗!
3,open_doc_prod38775855,197,197,20,0.10152284,港珠澳大桥隧道有46米深，万一漏水怎么逃生?看完实在佩服
4,20815k0K3kyHmZ,194,194,29,0.14948454,"邓超演唱《娘娘我错了》,“娘娘”惊喜登台他懵了!明星蒙圈集锦"
5,20816e0Q1gbo4w,192,192,92,0.47916667,龟头敏感早泄射精快怎么办，陈梅医生分享4个原因和解决方法
6,20815g0K3TyoOw,186,186,27,0.14516129,影视：穷小子混成亿万总裁，参加同学聚会偶遇初恋，好戏要开始
7,20527i0K43HFxB,178,178,25,0.14044944,假如越狱分段位，白展堂直接封神：老邢，再不开门我就出去了
8,20809c0R46SwzX,150,150,23,0.15333333,张怡宁对阵福原爱，一旁的教练急得不行：你让两球啊，她都要哭了
9,20814j043wYJx3,150,150,8,0.05333333,过气演员入住鬼宅，撞上阴兵借道，只能贴着墙根屏住呼吸


In [120]:
top_recall(df, 17)

810
0.22784810126582278


,docid,count,shows,clicks,ctr,title
0,2081600K4o4SgA,505,505,150,0.29702970,趁着你老婆没看见，赶紧看完吧。带上耳机最佳
1,20815d043j43Dg,382,382,121,0.31675393,恐怖片：小伙将自己化妆成死尸，本想吓唬回家的室友，结果救了自己一命
2,20815i0K0Dky56,255,255,85,0.33333333,难得一见激烈战斗蟒蛇发飙张开大嘴狠咬狮子见激烈战斗!
3,20816e0Q1gbo4w,239,239,124,0.51882845,龟头敏感早泄射精快怎么办，陈梅医生分享4个原因和解决方法
4,20815k0K3kyHmZ,202,202,31,0.15346535,"邓超演唱《娘娘我错了》,“娘娘”惊喜登台他懵了!明星蒙圈集锦"
5,open_doc_prod38775855,199,199,13,0.06532663,港珠澳大桥隧道有46米深，万一漏水怎么逃生?看完实在佩服
6,20809c0R46SwzX,198,198,14,0.07070707,张怡宁对阵福原爱，一旁的教练急得不行：你让两球啊，她都要哭了
7,20814g0Q0odUdB,182,182,12,0.06593407,中国将上万条“大鱼”养在三峡，如今变成了什么样?看完发出赞叹
8,20815g0K3TyoOw,155,155,38,0.24516129,影视：穷小子混成亿万总裁，参加同学聚会偶遇初恋，好戏要开始
9,open_doc_prod38511337,151,151,31,0.20529801,农民发现自家麦田被毁，用无人机观察后，暗自庆幸自己没有进去


In [121]:
x = df.docid.value_counts().head(20).reset_index()

In [122]:
x = x.rename(columns={'index': 'docid', 'docid': 'count'})

In [123]:
x = x.merge(df_doc, on='docid', how='left')
x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
x

,docid,count,shows,clicks,ctr,tdur,clicks2,mdur,title
0,2081600K4o4SgA,8142,8142,2487,0.30545321,83004,2336,35,趁着你老婆没看见，赶紧看完吧。带上耳机最佳
1,20815d043j43Dg,7347,7347,2015,0.27426160,219718,1808,121,恐怖片：小伙将自己化妆成死尸，本想吓唬回家的室友，结果救了自己一命
2,20815i0K0Dky56,5134,5134,1661,0.32352941,451138,1499,300,难得一见激烈战斗蟒蛇发飙张开大嘴狠咬狮子见激烈战斗!
3,20816e0Q1gbo4w,4730,4730,2212,0.46765328,222101,2064,107,龟头敏感早泄射精快怎么办，陈梅医生分享4个原因和解决方法
4,20815k0K3kyHmZ,3819,3819,546,0.14296936,55122,455,121,"邓超演唱《娘娘我错了》,“娘娘”惊喜登台他懵了!明星蒙圈集锦"
5,20815g0K3TyoOw,3671,3671,690,0.18795968,209427,607,345,影视：穷小子混成亿万总裁，参加同学聚会偶遇初恋，好戏要开始
6,open_doc_prod38775855,3527,3527,365,0.10348738,16190,305,53,港珠澳大桥隧道有46米深，万一漏水怎么逃生?看完实在佩服
7,20809c0R46SwzX,3242,3242,438,0.13510179,27822,395,70,张怡宁对阵福原爱，一旁的教练急得不行：你让两球啊，她都要哭了
8,20527i0K43HFxB,3164,3164,380,0.12010114,70379,337,208,假如越狱分段位，白展堂直接封神：老邢，再不开门我就出去了
9,20814j043wYJx3,2914,2914,231,0.07927248,56700,188,301,过气演员入住鬼宅，撞上阴兵借道，只能贴着墙根屏住呼吸


In [124]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [125]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [126]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [127]:
mean_duration = df[df.dur > 0].dur.mean()

In [128]:
mean_duration

139.79577574157477

In [129]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [130]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [131]:
cold_mids = set(df[df.cold==True].mid.values)

In [132]:
len(set(df.mid.values))

90584

In [133]:
len(cold_mids)

2748

In [134]:
df[df.docid=='20717c0K2Ouv1h'].dur

214720      0
233797      0
244366      0
259489      0
295649      0
414260    427
603003      0
680575    427
681836      0
800070      0
Name: dur, dtype: int64

In [135]:
df.abtestid

0          4
1         12
2         19
3          2
4          7
          ..
986740     7
986741    13
986742    12
986743    12
986744    16
Name: abtestid, Length: 986745, dtype: int64

In [136]:
def debug(abid, mid=None, cold=False):
  while True:
    df_ = df[df.abtestid==abid]
    if mid is None:
      if not cold:
        mid = df_.mid.sample(1).values[0]
      else:
        mid = df_[df_.mid.isin(cold_mids)].mid.sample(1).values[0]
    cols = ['position', 'title', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'shows', 'clicks', 'ctr', 'mdur', 'video_time', 'docid']
  #   if 'hour' in df.columns:
  #     cols += ['hour']
    d = df_[df_.mid==mid]
    print('mid:', mid, 'cold:', d.mid.values[0] in cold_mids, 'user_active:', d.user_active.values[0])
    start_time = d.impression_time.min()
    d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
    cols += ['time']
    d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#     d = d.sort_values(['impression_time', 'pred'], ascending=[True, False])[cols].reset_index(drop=True)
#     return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
    if d.dur.max() > 0:
      break
    else:
      mid = None
  return d

In [137]:
# mkywuen
debug(5,"c4421593426396019|bbd6f5")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [139]:
debug(5)

mid: e5b1A000007461FC7D cold: False user_active: 0
mid: ad33866546043126593 cold: False user_active: 0
mid: fc8d862279037964076 cold: False user_active: 2


,position,title,dur,ori_lr_score,pred,pred_click,pred_dur,shows,clicks,ctr,mdur,video_time,docid,time
0,0,张杰谢娜搞笑改编对唱《伤不起》来看看谁吹的过谁。,0,0.34255860,0.16560536,0.25720024,0.07311294,1,0,0.00000000,0,52,20814m0K0i4Op7,0
1,2,谢娜与专业舞者现场PK蒙古舞 丝毫不逊色,0,0.32684463,0.17182453,0.19372404,0.13750899,4,0,0.00000000,0,100,20816m0K29Ph95,0
2,4,吴昕“懒”出新境界，何炅间接吐槽：厨房里的摄像机都没开过吧!,93,0.32647806,0.17570404,0.18459278,0.16031629,1,1,1.00000000,93,173,20807j0K4ohTDO,0
3,6,谁是大歌神：许茹芸演唱《独角戏》优美的声音，韵味十足,0,0.31805703,0.17733729,0.16113275,0.21187869,1,0,0.00000000,0,287,20816h044yIGuK,0
4,0,锅碗瓢盆的现状如何?张继科把盆妹当闺女养，小碗每天和猪争宠,214,0.33314908,0.18683865,0.17957210,0.20112279,9,3,0.33333333,117,214,20811i0K4GgJx7,10


In [ ]:
debug(8)

In [ ]:
len(set(df.impression.values)) 

In [ ]:
len(set(df.mid.values))

In [ ]:
len(set(df.impression.values))  / len(set(df.mid.values))

In [ ]:
len(set(df[df.dur!=0].mid.values))

In [ ]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))